# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split

# Import Data

In [6]:
df = pd.read_csv('../../res/model_data/batemo_model_data.csv')
df = df.drop('SOH', axis=1) # Remove SOH from input data

# add moving averages of V, I, SOC, and T_surface to the input data
df['V_ma'] = df['V'].rolling(window=10).mean()
df['I_ma'] = df['I'].rolling(window=10).mean()
df['T_surf_ma'] = df['T_surf'].rolling(window=10).mean()

# drop the first 10 rows of the data since the moving averages are NaN for the first 10 rows
df = df.dropna()

df

,V,I,SOC,T_surf,V_ma,I_ma,T_surf_ma
9,4.146317,-9.736763,100.000000,25.000000,4.182662,-3.144607,25.000000
10,4.134066,-11.958543,100.000000,25.000000,4.176069,-4.340462,25.000000
11,4.122716,-14.017129,100.000000,25.000000,4.168465,-5.719655,25.000000
12,4.097001,-18.680679,100.000000,25.000000,4.158413,-7.542807,25.000000
13,4.075919,-22.503957,100.000000,25.000000,4.146375,-9.726015,25.000000
...,...,...,...,...,...,...,...
1092445,4.407437,40.000000,91.913889,99.066487,4.396315,40.000000,97.700077
1092446,4.408779,40.000000,92.140238,99.220353,4.398986,40.000000,98.037985
1092447,4.410138,40.000000,92.366586,99.374334,4.401460,40.000000,98.347912
1092448,4.413065,40.000000,92.872383,99.718690,4.403919,40.000000,98.648728


# Train Test Split

In [7]:
# split data to be 80% training and 20% testing using sklearn
train, test = train_test_split(df, test_size=0.2)

# define the input vectors and the corresponding labels 
# SOC from the input data is our corresponding label Y and all other data is the input features X
X_train = train.drop('SOC', axis=1)
Y_train = train['SOC']
X_test = test.drop('SOC', axis=1)
Y_test = test['SOC']

# Fit Model

In [8]:
# create an LSTM to train our model for finding SOC 
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model.summary()
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), verbose=2)

C:\Users\14845\Documents\ECE1896\venv_laptop\lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)                │     10,400 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 1)                 │         51 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


KeyboardInterrupt: 

# Test Model / Predict

In [38]:
# print accuracy
stats = model.evaluate(X_test, Y_test, return_dict=True)

4581/4581 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 9.6662e-04 - loss: 0.9400


In [39]:
# Print Accuracy and model statistics summary
print(f'Accuracy: {stats.get("accuracy")}')

Accuracy: 0.0010439339093863964


# Save Model and Export (Optimize for Size)